In [3]:
import argparse
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
DISCOVERY_URI = ('https://analyticsreporting.googleapis.com/$discovery/rest')
KEY_FILE_LOCATION = 'C:\\Users\\yuwei\\Documents\\CS565600\\cup01\\My First Project-2d655c2904ac.p12'
SERVICE_ACCOUNT_EMAIL = 'stone-1000@eco-channel-257005.iam.gserviceaccount.com'
VIEW_ID = '204660894'

In [4]:
def initialize_analyticsreporting():
  """Initializes an analyticsreporting service object.
  Returns:
    analytics an authorized analyticsreporting service object.
  """
  credentials = ServiceAccountCredentials.from_p12_keyfile(
    SERVICE_ACCOUNT_EMAIL, KEY_FILE_LOCATION, scopes=SCOPES)
  http = credentials.authorize(httplib2.Http())
  # Build the service object.
  analytics = build('analytics', 'v4', http=http, discoveryServiceUrl=DISCOVERY_URI)
  return analytics
def get_report(analytics):
  # Use the Analytics Service Object to query the Analytics Reporting API V4.
  return analytics.reports().batchGet(
        # Get sessions number from the last 7 days
#       body={
#         'reportRequests': [
#         {
#           'viewId': VIEW_ID,
#           'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
#           'metrics': [{'expression': 'ga:sessions'}]
#         }]
#       }
      # Get posts from last 7 days
      body={
          'reportRequests': [
              {
                  'viewId': VIEW_ID,
                  'dateRanges': [{'startDate': '7daysAgo', 'endDate': 'today'}],
                  'metrics': [
                      {'expression': 'ga:pageviews'},
                      {'expression': 'ga:uniquePageviews'},
                      {'expression': 'ga:timeOnPage'},
                      {'expression': 'ga:bounces'},
                      {'expression': 'ga:entrances'},
                      {'expression': 'ga:exits'}
                  ],
                  "dimensions": [
                      {"name": "ga:pagePath"}
                  ],
                  "orderBys": [
                      {"fieldName": "ga:pageviews", "sortOrder": "DESCENDING"}
                  ]
              }
          ]
      }
  ).execute()
def print_response(response):
  """Parses and prints the Analytics Reporting API V4 response"""
  for report in response.get('reports', []):
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    rows = report.get('data', {}).get('rows', [])
    for row in rows:
      dimensions = row.get('dimensions', [])
      dateRangeValues = row.get('metrics', [])
      for header, dimension in zip(dimensionHeaders, dimensions):
        print ( header + ': ' + dimension )
      for i, values in enumerate(dateRangeValues):
        print ('Date range (' + str(i) + ')' )
        for metricHeader, value in zip(metricHeaders, values.get('values')):
          print ( metricHeader.get('name') + ': ' + value )
def main():
  analytics = initialize_analyticsreporting()
  response = get_report(analytics)
  print_response(response)
if __name__ == '__main__':
  main()

HttpError: <HttpError 403 when requesting https://analyticsreporting.googleapis.com/v4/reports:batchGet?alt=json returned "Analytics Reporting API has not been used in project 433788882574 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/analyticsreporting.googleapis.com/overview?project=433788882574 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.". Details: "[{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Google developers console API activation', 'url': 'https://console.developers.google.com/apis/api/analyticsreporting.googleapis.com/overview?project=433788882574'}]}]">